In [3]:
def readfile(filename):
    file = open(filename, 'rt')
    original_text = file.read()
    file.close()
    
    return original_text

In [111]:
def writefile(filename, content):
    file = open(filename, 'w')
    file.write(str(content))
    file.close()

In [93]:
import re
def preprocess_data(text):
    # remove punctuations !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    punctuation_list = ["!","\"","#","$","%","&","'","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","\\","]","^","_","`","~","–","€","•","«","»","’","“","”","£"]
    for punctuation in punctuation_list:
        text = text.replace(punctuation, "")

#     remove numbers
    RE_NUMBERS = re.compile('[0-9]+')
    text = RE_NUMBERS.sub(r'', text)

    # remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    text = RE_EMOJI.sub(r'', text) # ⛈ corcra❄ gorm❄

    # substitute multiple spaces with a single space
    text = re.sub(' +', ' ', text)
    
    # trim leading and trailing spaces
    text = text.strip()

    # normalize text to lower case
    text = text.lower()
    
    return text

In [9]:
vocab = ["a","á","ais","áis","aisti","aistí","ait","áit","ar","ár","arsa","ársa","ban","bán","cead","céad","chas","chás","chuig","chúig","dar","dár","do","dó","gaire","gáire","i","í","inar","inár","leacht","léacht","leas","léas","mo","mó","na","ná","os","ós","re","ré","scor","scór","te","té","teann","téann","thoir","thóir"]
vocab_label_map = {"a": 0, "á": 1, "ais": 2, "áis": 3, "aisti": 4, "aistí": 5, "ait": 6, "áit": 7, "ar": 8, "ár": 9, "arsa": 10, "ársa": 11, "ban": 12, "bán": 13, "cead": 14, "céad": 15, "chas": 17, "chás": 18, "chuig": 19, "chúig": 20, "dar": 21, "dár": 22, "do": 23, "dó": 24, "gaire": 25, "gáire": 26, "i": 27, "í": 28, "inar": 29, "inár": 30, "leacht": 31, "léacht": 32, "leas": 33, "léas": 34, "mo": 35, "mó": 36, "na": 37, "ná": 38, "os": 39, "ós": 40, "re": 41, "ré": 42, "scor": 43, "scór": 44, "te": 45, "té": 46, "teann": 47, "téann": 48, "thoir": 49, "thóir": 50}

In [13]:
from sklearn.model_selection import train_test_split

text = readfile('cleaned_train_2.txt')

corpus = text.splitlines()

print("corpus len = " + str(len(corpus)))

X = []
y = []
for sentence in corpus:
    for v in vocab:
        if(v in sentence):
            X.append(sentence)
            y.append(vocab_label_map[v]) # label as numbers

print('X len = ' + str(len(X)))
print('y len = ' + str(len(y)))
            
X_train, X_test = train_test_split(X, test_size=0.2, random_state=69) 
y_train, y_test = train_test_split(y, test_size=0.2, random_state=69) 

corpus len = 456613
X len = 4159317
y len = 4159317


In [14]:
# import numpy as np

# predicted = model.predict(X_test)

# print(np.mean(predicted == y_test))  

In [113]:
from sklearn.feature_extraction.text import CountVectorizer
# token_re = '[(a)|(á)|(ais)|(áis)|(aisti)|(aistí)|(ait)|(áit)|(ar)|(ár)|(arsa)|(ársa)|(ban)|(bán)|(cead)|(céad)|(chas)|(chás)|(chuig)|(chúig)|(dar)|(dár)|(do)|(dó)|(gaire)|(gáire)|(i)|(í)|(inar)|(inár)|(leacht)|(léacht)|(leas)|(léas)|(mo)|(mó)|(na)|(ná)|(os)|(ós)|(re)|(ré)|(scor)|(scór)|(te)|(té)|(teann)|(téann)|(thoir)|(thóir)]'
# count_vect = CountVectorizer(analyzer='word',ngram_range=(1,2),token_pattern=token_re)
count_vect = CountVectorizer(analyzer='word',ngram_range=(2,2))
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3327453, 2453280)

In [114]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1)
model = clf.fit(X_train_counts, y_train)

In [115]:
test_sentences = readfile('test.txt')
test_sentences = preprocess_data(test_sentences)
test_sentences = test_sentences.splitlines()
print(len(test_sentences))

test_choice_1_sentences = [];
test_choice_2_sentences = [];

test_word_choice_1 = []
test_word_choice_2 = []

for s in test_sentences:
    s_idx = s.find("{")
    e_idx = s.find("}")
    if(s_idx == -1 | e_idx == -1):
        continue
    
    word_choice = s[(s_idx+1):e_idx] # "thoir|thóir"

    if word_choice:
        [word_choice_1, word_choice_2] = word_choice.split('|'); # word_choice_1 = "thoir" and word_choice_2 = "thóir"
        
        placeholder = s[(s_idx):e_idx+1]
        s1 = s.replace(placeholder, word_choice_1)
        s2 = s.replace(placeholder, word_choice_1)
        
        test_choice_1_sentences.append(s1)
        test_choice_2_sentences.append(s2)
        
        test_word_choice_1.append(word_choice_1)
        test_word_choice_2.append(word_choice_1)

20001


In [116]:
X_test_1_counts = count_vect.transform(test_choice_1_sentences)
X_test_2_counts = count_vect.transform(test_choice_2_sentences)
X_test_1_counts.shape
# print(clf.predict(X_test_counts))

In [119]:
test_1_probs = model.predict_proba(X_test_1_counts).tolist()
test_2_probs = model.predict_proba(X_test_2_counts).tolist()

answer_list = ["Id,Expected"]

for i in range(len(test_word_choice_1)):
    w1 = test_word_choice_1[i]
    w2 = test_word_choice_1[i]
    
    label_w1 = vocab_label_map[w1]
    label_w2 = vocab_label_map[w2]
    
    p1 = test_1_probs[i][label_w1]
    p2 = test_2_probs[i][label_w2]
    print(p1, p2)
    
    normalized_p1 = p1 / (p1+p2)
    normalized_p2 = p2 / (p1+p2)
    print(normalized_p1)
    answer_list.append("{},{:.20f}".format(i+1, normalized_p1))

answer = "\n".join(answer_list)

0.1298349572014497 0.1298349572014497
0.5
0.04368861595714276 0.04368861595714276
0.5
0.3589178970921784 0.3589178970921784
0.5
0.076253211794369 0.076253211794369
0.5
0.10959493642735155 0.10959493642735155
0.5
0.2510026158399944 0.2510026158399944
0.5
0.1750609962385074 0.1750609962385074
0.5
0.13021519564026138 0.13021519564026138
0.5
0.15761371275923805 0.15761371275923805
0.5
0.3496679851920898 0.3496679851920898
0.5
0.1466427494782082 0.1466427494782082
0.5
0.2199565329955222 0.2199565329955222
0.5
0.12784809754954415 0.12784809754954415
0.5
0.16506822274441957 0.16506822274441957
0.5
0.1819823366615471 0.1819823366615471
0.5
0.008709081498705999 0.008709081498705999
0.5
0.14421604727182927 0.14421604727182927
0.5
0.15054458346398464 0.15054458346398464
0.5
0.054856463893763134 0.054856463893763134
0.5
0.052641770252099096 0.052641770252099096
0.5
0.6925385617616554 0.6925385617616554
0.5
0.15796095581598282 0.15796095581598282
0.5
0.1191083887529967 0.1191083887529967
0.5
0.3101

0.5
0.16857126082817386 0.16857126082817386
0.5
0.018988679932904824 0.018988679932904824
0.5
0.14856061820431618 0.14856061820431618
0.5
0.17837856161471544 0.17837856161471544
0.5
0.19252756904355323 0.19252756904355323
0.5
0.1436526040866683 0.1436526040866683
0.5
0.34201656725256 0.34201656725256
0.5
0.09834073788136533 0.09834073788136533
0.5
0.12973904467528824 0.12973904467528824
0.5
0.7164819493722253 0.7164819493722253
0.5
0.25385023581632277 0.25385023581632277
0.5
0.2199724454732274 0.2199724454732274
0.5
0.16785351518121794 0.16785351518121794
0.5
0.1207599601645 0.1207599601645
0.5
0.20524837481920563 0.20524837481920563
0.5
0.17557436784699557 0.17557436784699557
0.5
0.00024295949969757819 0.00024295949969757819
0.5
0.16977845587216717 0.16977845587216717
0.5
0.17109369004422148 0.17109369004422148
0.5
3.843593076374099e-07 3.843593076374099e-07
0.5
0.06576797728078712 0.06576797728078712
0.5
0.18223502666345356 0.18223502666345356
0.5
0.17155828969185746 0.17155828969185

0.00109757427718849 0.00109757427718849
0.5
3.277709280662087e-05 3.277709280662087e-05
0.5
0.0005339880210037494 0.0005339880210037494
0.5
0.003313037659788784 0.003313037659788784
0.5
0.005736730545075234 0.005736730545075234
0.5
0.00012200777237541112 0.00012200777237541112
0.5
0.023309781340869473 0.023309781340869473
0.5
0.002562342084368173 0.002562342084368173
0.5
0.027256281818167725 0.027256281818167725
0.5
3.7082817540498185e-05 3.7082817540498185e-05
0.5
0.025839479271002448 0.025839479271002448
0.5
0.0011780773806330102 0.0011780773806330102
0.5
0.013892645270070579 0.013892645270070579
0.5
2.9570436932983135e-07 2.9570436932983135e-07
0.5
0.00699488186616183 0.00699488186616183
0.5
0.33668476235598954 0.33668476235598954
0.5
0.00016164734597115402 0.00016164734597115402
0.5
9.256021282663896e-05 9.256021282663896e-05
0.5
0.8935806187154618 0.8935806187154618
0.5
0.05445401209263286 0.05445401209263286
0.5
0.1870158084653778 0.1870158084653778
0.5
0.004795511383023919 0.004

0.5
2.1252441382580743e-14 2.1252441382580743e-14
0.5
2.127859452086352e-12 2.127859452086352e-12
0.5
5.323753515496627e-10 5.323753515496627e-10
0.5
0.00010792484937921356 0.00010792484937921356
0.5
3.542365063385209e-07 3.542365063385209e-07
0.5
2.2717667830022064e-22 2.2717667830022064e-22
0.5
5.883194672537568e-18 5.883194672537568e-18
0.5
1.5544508441583437e-42 1.5544508441583437e-42
0.5
0.7052148875387153 0.7052148875387153
0.5
7.988700675804741e-19 7.988700675804741e-19
0.5
1.1052442470957362e-06 1.1052442470957362e-06
0.5
5.2128024923312284e-06 5.2128024923312284e-06
0.5
1.8637512471997917e-05 1.8637512471997917e-05
0.5
4.483006381252204e-16 4.483006381252204e-16
0.5
4.890120470346647e-06 4.890120470346647e-06
0.5
9.27354487474233e-30 9.27354487474233e-30
0.5
1.2468007105731702e-13 1.2468007105731702e-13
0.5
1.968537624694243e-17 1.968537624694243e-17
0.5
3.4061496106861373e-16 3.4061496106861373e-16
0.5
4.35840730181191e-22 4.35840730181191e-22
0.5
2.4558146351352664e-16 2.455

0.0004103261957781448 0.0004103261957781448
0.5
0.0142459194334728 0.0142459194334728
0.5
0.001908022978873637 0.001908022978873637
0.5
1.4627441725467524e-12 1.4627441725467524e-12
0.5
0.03434595730350626 0.03434595730350626
0.5
0.09079500101032964 0.09079500101032964
0.5
1.1530925182618376e-05 1.1530925182618376e-05
0.5
6.283741638369981e-05 6.283741638369981e-05
0.5
0.00024354056588720514 0.00024354056588720514
0.5
0.00265205478647518 0.00265205478647518
0.5
6.923366482192404e-08 6.923366482192404e-08
0.5
0.2197764316334217 0.2197764316334217
0.5
0.01678543936326575 0.01678543936326575
0.5
1.706920180924253e-13 1.706920180924253e-13
0.5
0.00758147700295899 0.00758147700295899
0.5
1.6797959654527873e-05 1.6797959654527873e-05
0.5
6.300167883254247e-07 6.300167883254247e-07
0.5
0.001299009156409654 0.001299009156409654
0.5
0.06563283986924444 0.06563283986924444
0.5
0.02083818290071042 0.02083818290071042
0.5
0.016571751447967215 0.016571751447967215
0.5
0.7314622002035196 0.731462200

...

0.5
6.223502916671089e-32 6.223502916671089e-32
0.5
2.2384944739936092e-10 2.2384944739936092e-10
0.5
7.950732196056691e-21 7.950732196056691e-21
0.5
1.2442832486188157e-12 1.2442832486188157e-12
0.5
7.237760647304739e-10 7.237760647304739e-10
0.5
6.721146112213553e-25 6.721146112213553e-25
0.5
4.468785490399981e-13 4.468785490399981e-13
0.5
2.8421927896676353e-06 2.8421927896676353e-06
0.5
4.421827700366465e-29 4.421827700366465e-29
0.5
7.780889285436489e-15 7.780889285436489e-15
0.5
0.03846907901263176 0.03846907901263176
0.5
1.8876883399586612e-06 1.8876883399586612e-06
0.5
1.2727284184654995e-19 1.2727284184654995e-19
0.5
1.3662660700192132e-08 1.3662660700192132e-08
0.5
1.8453791651533132e-06 1.8453791651533132e-06
0.5
8.290570222305907e-26 8.290570222305907e-26
0.5
1.204799512122974e-07 1.204799512122974e-07
0.5
3.56459722096185e-08 3.56459722096185e-08
0.5
1.9063372174676394e-33 1.9063372174676394e-33
0.5
3.578133388744977e-58 3.578133388744977e-58
0.5
3.513201184105866e-21 3.51

5.788421876753072e-07 5.788421876753072e-07
0.5
5.606021440371305e-16 5.606021440371305e-16
0.5
9.874977877706034e-05 9.874977877706034e-05
0.5
0.00034730935514795436 0.00034730935514795436
0.5
1.2173418925802994e-06 1.2173418925802994e-06
0.5
0.00019868415792186625 0.00019868415792186625
0.5
4.756921774551767e-06 4.756921774551767e-06
0.5
0.022152793315152127 0.022152793315152127
0.5
6.81018550544225e-08 6.81018550544225e-08
0.5
1.0880077873897958e-12 1.0880077873897958e-12
0.5
1.4724168725164586e-05 1.4724168725164586e-05
0.5
1.0896093386348527e-11 1.0896093386348527e-11
0.5
3.3126570117659257e-09 3.3126570117659257e-09
0.5
1.654011515419772e-12 1.654011515419772e-12
0.5
0.09390695212524573 0.09390695212524573
0.5
6.742878972298097e-11 6.742878972298097e-11
0.5
9.087252697962299e-05 9.087252697962299e-05
0.5
2.128878345584871e-09 2.128878345584871e-09
0.5
2.052324811318624e-05 2.052324811318624e-05
0.5
1.4757682681392776e-05 1.4757682681392776e-05
0.5
0.001157198274484415 0.001157198

0.5
7.251795643691804e-11 7.251795643691804e-11
0.5
1.2253851805354632e-08 1.2253851805354632e-08
0.5
6.094045810084374e-27 6.094045810084374e-27
0.5
4.3358503572823284e-08 4.3358503572823284e-08
0.5
3.343334389976451e-12 3.343334389976451e-12
0.5
7.756207379788691e-21 7.756207379788691e-21
0.5
0.00022523342154481826 0.00022523342154481826
0.5
7.206769758553618e-11 7.206769758553618e-11
0.5
0.0006461437049574944 0.0006461437049574944
0.5
1.293990178034703e-08 1.293990178034703e-08
0.5
7.343857641112976e-13 7.343857641112976e-13
0.5
4.168745040959479e-12 4.168745040959479e-12
0.5
8.507929954896943e-07 8.507929954896943e-07
0.5
4.7770608900696566e-17 4.7770608900696566e-17
0.5
2.3569766279517845e-09 2.3569766279517845e-09
0.5
1.585630977061108e-12 1.585630977061108e-12
0.5
3.3502260799049075e-08 3.3502260799049075e-08
0.5
7.101184923135132e-31 7.101184923135132e-31
0.5
3.129098026147548e-11 3.129098026147548e-11
0.5
3.257927866430885e-12 3.257927866430885e-12
0.5
6.70537097087531e-20 6.7

3.344208653978689e-25 3.344208653978689e-25
0.5
6.8707696929563365e-18 6.8707696929563365e-18
0.5
1.2403025544580424e-07 1.2403025544580424e-07
0.5
7.401163938570349e-09 7.401163938570349e-09
0.5
1.216345676657009e-10 1.216345676657009e-10
0.5
2.1013347726804206e-09 2.1013347726804206e-09
0.5
2.2131902472036493e-06 2.2131902472036493e-06
0.5
3.171065109917573e-25 3.171065109917573e-25
0.5
8.740988348299721e-07 8.740988348299721e-07
0.5
0.00838503031561531 0.00838503031561531
0.5
1.6831720389061724e-32 1.6831720389061724e-32
0.5
6.050790045920405e-09 6.050790045920405e-09
0.5
1.5042734357767374e-20 1.5042734357767374e-20
0.5
0.005167401362308318 0.005167401362308318
0.5
2.8725793135613464e-09 2.8725793135613464e-09
0.5
6.446170925766871e-11 6.446170925766871e-11
0.5
3.171740910844677e-06 3.171740910844677e-06
0.5
8.78247594508704e-05 8.78247594508704e-05
0.5
9.064071253014452e-13 9.064071253014452e-13
0.5
0.9375764438002891 0.9375764438002891
0.5
1.6569334634509366e-09 1.656933463450936

0.5
7.760536758847732e-05 7.760536758847732e-05
0.5
4.34069298894877e-12 4.34069298894877e-12
0.5
3.5090586146636974e-10 3.5090586146636974e-10
0.5
8.765095478210149e-07 8.765095478210149e-07
0.5
1.5926373582563758e-14 1.5926373582563758e-14
0.5
3.2719989718971196e-19 3.2719989718971196e-19
0.5
6.614492370990222e-13 6.614492370990222e-13
0.5
1.3905957687185951e-13 1.3905957687185951e-13
0.5
3.014173878911822e-10 3.014173878911822e-10
0.5
0.0032621009723984543 0.0032621009723984543
0.5
5.51779528638136e-34 5.51779528638136e-34
0.5
4.062374201745719e-15 4.062374201745719e-15
0.5
2.2439575488177782e-11 2.2439575488177782e-11
0.5
5.053177934197227e-21 5.053177934197227e-21
0.5
1.8289165983758723e-28 1.8289165983758723e-28
0.5
1.6597560649572302e-12 1.6597560649572302e-12
0.5
2.843637588561748e-08 2.843637588561748e-08
0.5
1.1373634765940297e-15 1.1373634765940297e-15
0.5
6.800643308430344e-06 6.800643308430344e-06
0.5
6.932972321817148e-18 6.932972321817148e-18
0.5
3.174406827594539e-05 3.

In [112]:
writefile('submission_naive_bayes_add_0.5_smoothing_16.csv', answer)

In [ ]:
def generate_submission_bigram():
    test_sentences = readfile('mini_test.txt.txt').splitlines();

#     test_sentences = test_text.split("\n")
    answer_list = ["Id,Expected"]
    running_number = 1
   
        print(model.predict_proba(X_train_counts[0]))
        answer_list.append("{},{:.20f}".format(running_number, prob))
        running_number += 1
    answer = "\n".join(answer_list)
    return answer